# Comunicacion de datos

**NOTA: El ambiente de Jupyter Notebook no es el espacio ideal de programación donde codificar programas de comunicación. Esta guía es más un documento de referencia con los códigos a ser probados en un ambiente como Spyder o la consola de comandos.**

Se puede establecer comunicación de datos entre perifericos utilizando diferentes medios y tecnologías. Python tiene diferentes librerias para poder intercambiar datos con otros elementos, utilizando dos tecnologías de transmisión de datos:

* Serial (COM, UART, USB)
* Sockets (Ethernet, WiFi)

## Socket
Un socket es el elemento que utiliza un software para establacer una conexión de red basada en el protocolo IP (ya sea en version 4 o 6) y utilizando protocolos de red (ya sea TCP o UDP). Un socket es una estructura que esta conformada por dos elementos:

* Una direccion de red (IP)
* Un puerto de comunicación (TCP/UDP)

Los métdodos dispoibles en el modulo socket son:

* socket()
* bind()
* listen()
* accept()
* connect()
* connect_ex()
* send()
* recv()
* close()

Un socket se crea utilizando un objeto socket definido con la instrucción socket.socket() y especificando la versión de protocolo IP a utilizar y el protocolo de transmision a usar (TCP: socket.SOCK_STREAM, UDP: SOCK_DGRAM). En los ejemplos utilizaremos el protocolo TCP por ser más estable, confiable y seguro.

En el siguiente diagrama se resume el proceso de llamada a los sockets (socket API) y el flujo de datos TCP en la comunicación entre nodos (en un esquema cliente-servidor, es decir, el nodo "servidor" esta esperando las conexiones de los nodos "cliente" para atender sus requerimientos).

<img src="https://i.imgur.com/zmhwCSA.png" alt="Girl in a jacket" width="700" height="800">

## Echo "Hola Mundo
### Script servidor

In [1]:
import socket

In [ ]:
import socket

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)

# AF_INET: Address Family Intenet IP v4
# SOCK_STREAM: TCP Protocol

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    # Asocia un Host y Puerto a un Socket
    s.bind((HOST, PORT))
    # Coloca el socket en modo escucha
    s.listen()
    # Acepta conexiones entrantes
    conn, addr = s.accept()    # <- Blocking function (retorna socket, direccion)

    with conn:
        print('Conectado a', addr)
        while True:
            # Recibe datos en un buffer de 1024 bytes
            data = conn.recv(1024)
            if not data:
                break
            
            # Si hay datos en el buffer, broadcast...
            conn.sendall(data)
    

### Script cliente

In [2]:
import socket

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65432        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    # Utilizar el socket para conectarse a una direccion y puerto
    s.connect((HOST, PORT))
    # Enviar data binaria
    s.sendall(b'Hola mundo')
    # Recibir datos en un buffer de 1024
    data = s.recv(1024)

print('Recibido', repr(data))

ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

## Intrercambio de información (Cliente - Sercvidor)
Las comunicaciones en red se sirven de un modelo llamado Cliente-Servidor, donde los Clientes son los elementos que se 
comunicarán entre si, mientras que el Servidor será el intermediario en el intercambio de información entre los Clientes. Para esto sera necesario que el Servidor pueda recibir la información del Cliente y este enviarla de forma inmediata.

### Script Servidor

In [ ]:
import socket

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    
    try:
        with conn:
            print("Conectado a", addr)
            while True:
                data = conn.recv(1)
                
                if not data:
                    break
                else:
                    strData = data.decode('utf-8')
                    print(strData)
                          
    except KeyboardInterrupt:
        pass
    
print("Cerrando conexion")

### Script Cliente

In [ ]:
import socket
import time

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65432        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    
    for i in range(1, 11):
        strData = str(i)
        data = strData.encode('utf-8')
        s.send(data)
        
        time.sleep(1)

## Header de comunicación: Agregando encabezado a la transmisión para la señalización
En el ejemplo anterior, se puede ver que el servidor recibe los datos de forma inmediata 1 byte a la vez, lo que hace que se tomen los bytes de manera rapida pero será necesario recomponer el mensaje original. Para obtener velocidad y control del mensaje, lo ideal es que el buffer del Server será variable y su tamaño este en función de mensaje a ser enviado por el Cliente

Definiremos un protocolo de comunicación que incluirá un Header de 10 catacteres (bytes), en donde se especificará en número de caracteres (bytes) que serán enviados al Server.

### Script Servidor

In [ ]:
import socket

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)
HEADER = 10

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    
    try:
        with conn:
            print("Conectado a", addr)
            while True:
                data_len = conn.recv(HEADER)
                
                if not data_len:
                    break
                else:
                    data = conn.recv(int(data_len))
                    strData = data.decode('utf-8')
                    print(strData)
                          
    except KeyboardInterrupt:
        pass
    
print("\nCerrando conexion")

### Script Cliente

In [ ]:
import socket
import time

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65432        # The port used by the server
HEADER = 10

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    
    names = ['Maria',
             'Rodrigo',
             '',
             'Juan',
             'Nabucodonosor',
             'Sandro']
    
    for i in names:
        # FORMATO: <HEADER><DATA>
        strData = str(i)        
        data_len = str(len(strData))
        
        data = f"{data_len:<{HEADER}}{strData}".encode('utf-8')
        
        s.send(data)
        
        time.sleep(1)

## Serialización: el módulo `pickle`
El módulo `pickle` permite "serializar" un dato (mashalling), esto es convertír un objeto (como puede ser una lista, un diccionario, un arreglo, un archivo, etc) en una trama que tal forma que pueda enviarse por un canal de comunicación.

### Script Servidor

In [ ]:
import socket
import pickle

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)
HEADER = 10

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    +
    try:
        with conn:
            print("Conectado a", addr)
            while True:
                data_len = conn.recv(HEADER)
              
                if not data_len:
                    break
                else:
                    data = b''
                    data += conn.recv(int(data_len))
                    data_deserial = pickle.loads(data)
                    print(data_deserial)
                          
    except KeyboardInterrupt:
        pass
    
print("\nCerrando conexion")

### Script Cliente

In [ ]:
import socket
import pickle

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65432        # The port used by the server
HEADER = 10

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    
    names = [{'nombre': 'Elvio', 'apellido': 'Lado'}, 
             ('ene', 'feb', 'mar'), 
             1+3j]
    
    data_serial = pickle.dumps(names)   # dumps: vuelca a una trama binaria
    
    # FORMATO: <HEADER><DATA>
    data_len = str(len(data_serial))
        
    data = bytes(f"{data_len:<{HEADER}}", 'utf-8') + data_serial
    print(data)
    s.send(data)